# Пример агента для бронирования отелей и авиабилетов

Это решение поможет вам забронировать авиабилеты и отель. Сценарий: поездка из Лондон Хитроу (LHR) 20 февраля 2024 года в Нью-Йорк (JFK) с возвращением 27 февраля 2025 года, эконом-класс, только British Airways. Я хочу остановиться в отеле Hilton в Нью-Йорке, пожалуйста, предоставьте стоимость авиабилетов и отеля.


# Инициализация службы Azure AI Agent и получение информации о конфигурации из **.env**

### **.env**

Создайте файл .env

**.env** содержит строку подключения к службе Azure AI Agent, модель, используемую AOAI, а также соответствующий API Google API Search, ENDPOINT и другие параметры.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Имя развертывания модели вашей службы Azure AI Agent"

[**NOTE**] Вам потребуется модель с лимитом 100,000 токенов в минуту и лимитом 600 запросов в минуту.

  Модель можно получить в Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Строка подключения к проекту вашей службы Azure AI Agent"

  Строку подключения к проекту можно найти в обзоре проекта на экране портала AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Ваш ключ API SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Ваш ENDPOINT SERPAPI Search"

Чтобы получить имя развертывания модели и строку подключения к проекту службы Azure AI Agent, необходимо создать службу Azure AI Agent. Рекомендуется использовать [этот шаблон](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) для прямого создания. (***Примечание:*** Служба Azure AI Agent в настоящее время доступна только в ограниченных регионах. Рекомендуется ознакомиться с [этой ссылкой](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) для настройки региона.)

Агенту необходимо получить доступ к SERPAPI. Рекомендуется зарегистрироваться, используя [эту ссылку](https://serpapi.com/searches). После регистрации вы сможете получить уникальный ключ API и ENDPOINT.


# Настройка

Чтобы запустить этот ноутбук, убедитесь, что вы установили необходимые библиотеки, выполнив команду `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ваша версия Semantic Kernel должна быть не ниже 1.27.2.


Загрузите настройки и ресурсы вашего файла .env, пожалуйста, убедитесь, что вы добавили свои ключи и настройки, а также создали локальный файл .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Вход в Azure

Теперь вам нужно войти в Azure. Откройте терминал и выполните следующую команду:

```bash
az login
```

Эта команда запросит ввод ваших учетных данных Azure, что позволит сервису Azure AI Agent работать корректно.


# Объяснение:
Это переменная, которая хранит ключ API для доступа к сервису API поисковой страницы (SERP). Ключ API — это уникальный идентификатор, используемый для аутентификации запросов, связанных с вашим аккаунтом.

Цель: Цель этой строки — сохранить ключ API в переменной, чтобы его можно было использовать для аутентификации запросов к сервису SERP API. Ключ API необходим для доступа к сервису и выполнения поисковых запросов.

Как получить ключ API для SERP: Чтобы получить ключ API для SERP, выполните следующие общие шаги на https://serpapi.com (точные шаги могут отличаться в зависимости от конкретного сервиса SERP API, который вы используете):

Выберите сервис SERP API: Существует несколько доступных сервисов SERP API, таких как SerpAPI, Google Custom Search JSON API и другие. Выберите тот, который лучше всего соответствует вашим потребностям.

Зарегистрируйтесь: Перейдите на сайт выбранного сервиса SERP API и зарегистрируйтесь. Возможно, вам потребуется предоставить базовую информацию и подтвердить адрес электронной почты.

Создайте ключ API: После регистрации войдите в свой аккаунт и перейдите в раздел API или на панель управления. Найдите опцию для создания или генерации нового ключа API.  
Скопируйте ключ API в ваш файл .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Объяснение:
BASE_URL: Это переменная, которая хранит базовый URL для конечной точки API SERP. Имя переменной BASE_URL принято использовать для обозначения того, что этот URL является отправной точкой для выполнения запросов к API.  
'https://serpapi.com/search':  

Это фактическая строка URL, назначенная переменной BASE_URL. Она представляет конечную точку для выполнения поисковых запросов с использованием API SERP.  

# Цель:
Цель этой строки — определить константу, которая содержит базовый URL для API SERP. Этот URL будет использоваться как отправная точка для построения запросов к API для выполнения поисковых операций.  

# Использование:
Определив базовый URL в переменной, вы можете легко использовать его повторно в своем коде всякий раз, когда вам нужно выполнить запросы к API SERP. Это делает ваш код более удобным для сопровождения и снижает риск ошибок, связанных с жестким кодированием URL в нескольких местах. Текущий пример — https://serpapi.com/search?engine=bing, который использует API поиска Bing. Различные API можно выбрать на https://Serpapi.com  


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Объяснение:

Здесь находится код вашего плагина.

Определение класса: `class BookingPlugin`: Определяет класс с именем BookingPlugin, который содержит методы для бронирования отелей и авиабилетов.

Метод бронирования отелей:

- `@kernel_function(description="booking hotel")`: Декоратор, описывающий функцию как ядровую функцию для бронирования отелей.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Определяет метод для бронирования отелей с аннотированными параметрами и типом возвращаемого значения.

Метод формирует словарь параметров для запроса на бронирование отеля и отправляет GET-запрос к API SERP. Он проверяет статус ответа и возвращает свойства отеля в случае успеха или None, если запрос не удался.

Метод бронирования авиабилетов:

- `@kernel_function(description="booking flight")`: Декоратор, описывающий функцию как ядровую функцию для бронирования авиабилетов.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Определяет метод для бронирования авиабилетов с аннотированными параметрами и типом возвращаемого значения.

Метод формирует словари параметров для запросов на вылет и обратный рейс и отправляет GET-запросы к API SERP. Он проверяет статус ответа и добавляет информацию о рейсе в результирующую строку в случае успеха или выводит сообщение об ошибке, если запрос не удался. Метод возвращает результирующую строку с информацией о рейсах.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Объяснение:
Импорт модулей: Импортируются необходимые модули для работы с учетными данными Azure, AI-агентом, содержимым сообщений чата, ролью автора и декоратором функций ядра.

Асинхронный контекстный менеджер: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Настраивается асинхронный контекстный менеджер для работы с учетными данными Azure и создания клиента AI-агента.

Имя агента и инструкции:
- `AGENT_NAME = "BookingAgent"`: Определяет имя агента.
- `AGENT_INSTRUCTIONS = """..."""`: Предоставляет подробные инструкции для агента о том, как обрабатывать запросы на бронирование.

Создание определения агента: `agent_definition = await client.agents.create_agent(...)`: Создает определение агента с указанной моделью, именем и инструкциями.

Создание AzureAI-агента: `agent = AzureAIAgent(...)`: Создает AzureAI-агента, используя клиента, определение агента и заданный плагин.

Создание потока: `thread: AzureAIAgentThread | None = None`: Создает поток для агента. Создание потока не является обязательным - если указано значение `None`, новый поток будет создан при первом вызове и возвращен как часть ответа.

Ввод данных пользователя: `user_inputs = ["..."]`: Определяет список пользовательских данных, которые агент должен обработать.

В блоке `finally` удаляются поток и агент для освобождения ресурсов.


# Аутентификация

Класс `DefaultAzureCredential` является частью Azure SDK для Python. Он предоставляет стандартный способ аутентификации с сервисами Azure. Класс пытается выполнить аутентификацию, используя несколько методов в определённом порядке, таких как переменные окружения, управляемая идентичность и учетные данные Azure CLI.

Асинхронные операции: Модуль aio указывает на то, что класс DefaultAzureCredential поддерживает асинхронные операции. Это означает, что вы можете использовать его с asyncio для выполнения неблокирующих запросов на аутентификацию.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
